# CS211: Data Privacy
## Homework 8

In [1]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

/tmp/ipykernel_1302/2770425378.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
# preserves epsilon-differential privacy
def above_threshold(query_results, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)
    
    for idx, q in enumerate(query_results):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q + nu_i >= T_hat:
            return idx
    return None

## Question 1 (20 points)

Implement a function `above_10000` that releases the **value** of the first query in a sequence of queries whose value is above 10000. Your function should have a **total** privacy cost equal to the privacy parameter $\epsilon$ passed in when it is called.

**Note**: this function (and the rest of the ones you'll define in this assignment) take a list of *query results* rather than the queries themselves (as we saw in class). This simplification makes your code a little bit simpler.

In [3]:
def above_10000(query_results, epsilon):
    t_hat = 10000 + np.random.laplace(loc=0, scale=2/epsilon)
    
    for idx, q in enumerate(query_results):
        noise = np.random.laplace(loc=0, scale=4/epsilon)
        if q + noise >= t_hat:
            return q + noise

    return -1

queries = adult['Marital Status'].value_counts()
print(f"above_10000 #1: {above_10000(queries, 100)}")
print(f"above_10000 #2: {above_10000(queries, 1)}")
print(f"above_10000 #3: {above_10000(queries, .01)}")

above_10000 #1: 14975.947678301522
above_10000 #2: 14978.927765745346
above_10000 #3: 14793.793406140809


In [4]:
# TEST CASE

results = [above_10000(queries, 1.0) for _ in range(20)]
print(np.mean(results))
assert np.mean(results) > 14900
assert np.mean(results) < 15000

14975.184392322588


## Question 2 (10 points)
In 2-3 sentences, argue informally (via the definition of the sparse vector technique, post-processing, and sequential composition), that your implementation of `above_10000` has a total privacy cost of $\epsilon$.

My implementation of above_10000 must have total privacy cost $\epsilon$ since only one noisy query is released. Although it is given a list of results and adds noise to them, the totaly privacy cost only accounts for the results we release and since the method will only release the result which passes our threshold test, it preserves a total privacy cost of epsilon.

## Question 3 (20 points)

Implement a function `bounded_all_above_10000` that releases the **value** of **$c$ queries** in a sequence of queries whose value is above 10000 (where $c$ is an analyst-provided parameter limiting the number of returned results). Your function should have a **total privacy cost** bounded by its parameter $\epsilon$.

In [5]:
def bounded_all_above_10000(query_results, c, epsilon):
    results = []
    i = 0
    epsilon_i = epsilon/c

    while i < len(query_results) and len(results) < c:
        x = above_10000(query_results[i:], epsilon_i)
        if x == -1:
            return results
        else:
            results.append(x)
            i += 1
            
    return results

queries = list(adult['Marital Status'].value_counts())
print(f"bounded_all_above_10000 #1: {bounded_all_above_10000(queries, 3, 100)}")
print(f"bounded_all_above_10000 #2: {bounded_all_above_10000(queries, 3, 1)}")
print(f"bounded_all_above_10000 #3: {bounded_all_above_10000(queries, 3, .01)}")

bounded_all_above_10000 #1: [14975.970034909462, 10683.1713146976]
bounded_all_above_10000 #2: [14982.420450951086, 10688.714410194092]
bounded_all_above_10000 #3: [14491.42374625046, 13459.6739073133]


In [6]:
# TEST CASE

results = [bounded_all_above_10000(queries, 2, 1.0)]
results_1 = [r[0] for r in results]
results_2 = [r[1] for r in results]

assert np.mean(results_1) > 14900
assert np.mean(results_1) < 15000
assert np.mean(results_2) > 10600
assert np.mean(results_2) < 10700

## Question 4 (10 points)

In 2-3 sentences, argue informally that your implementation of `bounded_all_above_10000` has privacy cost bounded by $\epsilon$.

The bounded_all_above_10000 method has a privacy cost $\epsilon$ since the total privacy budget is split into c ways. This means that c differentially privacy queries can be computed while still maintaining a total privacy budget of $\epsilon$.

## Question 5 (30 points)

Implement a function `mean_age` that computes the mean age of participants in the `adult_data` dataset. Your function should have a **total** privacy cost of $\epsilon$. It should work as follows:

1. Compute an *upper* clipping parameter based on the data
2. Clip the data using the clipping parameter
3. Use `laplace_mech` to release a differentially private mean of the clipped data

*Hint*: Use the sparse vector technique (`above_threshold`) to compute the clipping parameter. Consider using a sequence of queries that looks like `df.clip(lower=b, upper=0).sum() - df.clip(lower=b+1, upper=0).sum()`.

*Hint*: Be careful of sensitivities and set the scale of the noise accordingly!

In [7]:
bs = list(range(0, 200, 10))
df = adult['Age']

def mean_age(epsilon):
    epsilon_i = epsilon/3
    
    upper_bound = bs[above_threshold([(df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b+1).sum()) for b in bs], 0, epsilon_i)]
    return laplace_mech(df.clip(lower=0, upper=upper_bound).sum(), upper_bound, 
                        epsilon_i)/laplace_mech(len(df), 1, epsilon_i)
    
for epsilon in [0.001, 0.01, 0.1, 0.5, 1, 10]:
    print(f"epsilon: {epsilon}, mean age: {mean_age(epsilon)}")

epsilon: 0.001, mean age: 37.29460209674652
epsilon: 0.01, mean age: 38.28291170431183
epsilon: 0.1, mean age: 38.588232731766325
epsilon: 0.5, mean age: 38.48778801459797
epsilon: 1, mean age: 38.599803762933576
epsilon: 10, mean age: 38.58073402701857


In [8]:
# TEST CASE
results = [mean_age(1.0) for _ in range(20)]
assert np.mean(results) > 38
assert np.mean(results) < 39

## Question 6 (10 points)

In 3-5 sentences, describe your approach to implementing `mean_age` and argue informally that your implementation has privacy cost bounded by $\epsilon$.

My implementation of mean_age must have a total privacy cost of $\epsilon$ since there are 3 different queries occouring in order to produce the final differentially private mean. The first occours in the upper_bound variable declaration where the method above_0 is used to calculate a threshold. Next, to return the final result we run the laplace mech on both the sum and the length in order to compute the mean. Since our epsilon is divided evenly into 3 chunks, by sequential composition since we are running 3 differentially private mechanisms we will have a total privacy cost of epsilon

## Question 7 (20 points)

Write an algorithm to compute the maximum of a given column of the adult dataset. Your solution should:

1. Use the Sample & Aggregate technique to compute the maximum
2. Use AboveThreshold to set the clipping parameter(s) used in Sample & Aggregate

In [9]:
bs = [2**x for x in list(range(1, 50))]

def f(chunk):
    return chunk.max()

def saa_max(col, epsilon):
    df = adult[col]
    epsilon_i = epsilon/2 # Only running 2 differentially private mechs

    # split the data into chunks
    chunk_size = int(np.ceil(df.shape[0] / (len(df)/4)))
    chunks = [df[i:i+chunk_size] for i in range(0 , df.shape[0], chunk_size)]
    chunk_answers = [f(chunk) for chunk in chunks]

    # Find lower and upper bounds
    upper_bound = bs[above_threshold([(df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b+1).sum()) for b in bs], 0, epsilon_i)]
    
    # run the query on each chunk using above threshold
    clipped_answers = np.clip(chunk_answers, 0, upper_bound)

    # sum the clipped answers (sensitivity = b)
    sum_result = np.sum(clipped_answers)
    avg_result = sum_result / len(chunks)

    # add noise to the sum (scale = b /epsilon)
    noisy_answer = avg_result + laplace_mech((upper_bound / (len(chunks) * epsilon)), 1, epsilon_i)
    
    return noisy_answer

saa_max('Age', 1.0)

55.30508753357446

In [10]:
# TEST CASE
cols = ['Age', 'Capital Gain', 'fnlwgt']

for c in cols:
    true_val = adult[c].max()
    trials = [saa_max(c, 10.0) for _ in range(20)]
    errors = [pct_error(true_val, t) for t in trials]
    print('Median error for column ' + c + ':', np.median(errors))
    assert np.median(errors) > 0
    assert np.median(errors) < 100


Median error for column Age: 40.53116811669884
Median error for column Capital Gain: 95.9020052027509
Median error for column fnlwgt: 79.65786083349735


## Question 8 (10 points)

In 3-5 sentences, describe your approach to implementing `saa_max` and argue that your approach has total privacy cost bounded by $\epsilon$.

The first step to implementing saa_max is finding chunk sizes and creating those chunks. So I did that, splitting up and then computing the maximum value of each chunk. Next I computed the upper bound using the above threshold method we previously implemented, based on the given set of b's and clipped each chunk answer to those bounds. Finally, I computed the average answer of each chunk and added noise based on the scale of the clipping bounds. This whole function maintains a total privacy budget of epsilon since there are 2 differentially private mechanisms and each is given epsilon/2 as a privacy parameter. 